In [1]:
!pip list


import sys
print(sys.version)



# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Package                               Version               Editable project location
------------------------------------- --------------------- -------------------------
absl-py                               0.15.0
accelerate                            0.5.1
access                                1.1.3
affine                                2.3.0
aiobotocore                           2.1.2
aiohttp                               3.8.1
aioitertools                          0.10.0
aiosignal                             1.2.0
albumentations                        1.1.0
alembic                               1.7.7
allennlp                              2.9.1
altair                                4.2.0
annoy                                 1.17.0
ansiwrap                              0.8.4
anyio                                 3.4.0
apache-beam                           2.34.0
aplus                                 0.11.0
appdirs                               1.4.4
argcomplete                    

In [2]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        batch_input_shape=[batch_size, None]
    ))
    model.add(tf.keras.layers.LSTM(
        units=rnn_units,
        return_sequences=True,
        stateful=True, #look into this causing the issue potentially, originally was True
        recurrent_initializer=tf.keras.initializers.GlorotNormal() #used to be be tf.keras.initializers.GlorotNormal() before downgradingf
    )) # temp removed
    model.add(tf.keras.layers.Dense(vocab_size))
    
    return model

In [3]:
#imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # ML/DL
import keras
from keras.models import Sequential
from tensorflow.keras.layers import LSTM
from keras.layers import Activation, Dense, Dropout
import matplotlib.pyplot as plt
import tqdm
import pickle

#tf.compat.v1.enable_eager_execution()
import tensorflow_addons as tfa
#import coremltools as ct

In [4]:
try: #for recursive model training
    df = pd.read_csv(os.path.join('./WORKING'))
    print('Wokrgin')
except:
    df = pd.read_csv(os.path.join('../input/newencoded/encodedNew.csv'))
#OG DATAFRAME
df.head()

,d/l,pu,tpu,rpu,su,tsu,rsu,sq,tsq,rsq,bp,tbp,rbp
0,4,8,10,18,25,35,42,56,70,2,63,80,0
1,1,3,25,28,3,15,61,7,50,6,6,50,25
2,2,21,55,19,24,70,10,10,30,2,13,40,27
3,2,12,30,45,12,35,6,8,25,7,28,80,0
4,4,11,15,53,12,15,5,20,25,36,48,60,17


In [5]:

STOP_SIGN = "␣"   
allWorkoutsString = []
stringVocab = []

import random
def alterRange(pu,tpu): #THE MODEL DOESNT CONTAIN ALL THE VOCAB WE ARE ASKING IT TO, EITHER MAP TO CLOSEST VOCAB OR RETAIN THE MOBEL ON MORE POSSIBLE NUMBER
    mul = pu/tpu
    alterMul = mul + 0.1 #should be 0.03 but ill make it 0.5 for now
    pu = alterMul * tpu  
    return int(float(pu))


##NEW SHUFFLING TO FIX POOR EARLY PERFORMANCE
print(df.head())
df = df.sample(frac = 1)
print(df.head())
##NEW SHUFFLING TO FIX POOR EARLY PERFORMANCE

for i in range(len(df)):
    #df.iloc[i]["pu"] = alterRange(df.iloc[i]["pu"],df.iloc[i]["tpu"]) #right now it overall makes pu way more diffiuclt
    workout = (str(df.iloc[i]["d/l"]) + "-DL:~" + str(df.iloc[i]["pu"]) + "~" + str(df.iloc[i]["tpu"]) + "~" + str(df.iloc[i]["rpu"]) + "~" + str(df.iloc[i]["su"]) + "~" + str(df.iloc[i]["tsu"]) + "~" + str(df.iloc[i]["rsu"]) + "~" + str(df.iloc[i]["sq"]) + "~" + str(df.iloc[i]["tsq"]) + "~" + str(df.iloc[i]["rsq"]) + "~"  +  str(df.iloc[i]["bp"]) + "~" + str(df.iloc[i]["tbp"]) + "~" + str(df.iloc[i]["rbp"]) + "~" + "␣" )
    #allWorkoutsString.append(workout)
    stringVocab.append(workout)
      
        

df.to_csv("./WORKING")
#
#for z in range(64):          
allWorkoutsString.append("5-DL:~50~50~28~14~15~17~15~15~36~34~35~5~␣")
#
#    allWorkoutsString.append("1-DL:~20~25~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~43~55~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~30~45~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~25~30~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~50~55~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~18~20~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~8~10~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~55~60~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~32~40~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~37~45~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~14~15~22~45~55~0~5~25~48~2~10~47~␣")
#    allWorkoutsString.append("1-DL:~60~60~22~45~55~0~5~25~48~2~10~47~␣")
#    


#for j in range(30):
#   print(allWorkoutsString[j])



   d/l  pu  tpu  rpu  su  tsu  rsu  sq  tsq  rsq  bp  tbp  rbp
0    4   8   10   18  25   35   42  56   70    2  63   80    0
1    1   3   25   28   3   15   61   7   50    6   6   50   25
2    2  21   55   19  24   70   10  10   30    2  13   40   27
3    2  12   30   45  12   35    6   8   25    7  28   80    0
4    4  11   15   53  12   15    5  20   25   36  48   60   17
      d/l  pu  tpu  rpu  su  tsu  rsu  sq  tsq  rsq  bp  tbp  rbp
356     1  16   80    0   9   75    4   2   15   51  12   60    6
917     5  72   75    5  63   70    7  39   40   23  50   50   19
1185    5  60   65   15  69   70    6  53   55    8  15   15   54
143     4  54   70    4  16   20   59  35   45   25  33   45    1
1484    3  18   30   33  33   60   14  41   75    2  32   55    8


In [6]:
def decodeWorkout(input):    
    c = input.split("~")
    if(len(c) == 5):
        pushupStr = str(c[0] + "\n" + c[1] + " pushups for " + c[2] + " seconds with " + c[3] + " second rest ")
        outputStr = (pushupStr)
    elif(len(c) == 8):
        pushupStr = str(c[0] + "\n" + c[1] + " pushups for " + c[2] + " seconds with " + c[3] + " second rest ")
        situpStr = str("\n" + c[4] + " situps for " + c[5] + " seconds with " + c[6] + " second rest ")
        outputStr = (pushupStr + situpStr)
    elif(len(c)==11):
        pushupStr = str(c[0] + "\n" + c[1] + " pushups for " + c[2] + " seconds with " + c[3] + " second rest ")
        situpStr = str("\n" + c[4] + " situps for " + c[5] + " seconds with " + c[6] + " second rest ")
        squatStr = str("\n" + c[7] + " squats for " + c[8] + " seconds with " + c[9] + " second rest ")
        outputStr = (pushupStr + situpStr + squatStr)
    elif(len(c) == 14):
        pushupStr = str(c[0] + "\n" + c[1] + " pushups for " + c[2] + " seconds with " + c[3] + " second rest ")
        situpStr = str("\n" + c[4] + " situps for " + c[5] + " seconds with " + c[6] + " second rest ")
        squatStr = str("\n" + c[7] + " squats for " + c[8] + " seconds with " + c[9] + " second rest ")
        burpeeStr = str("\n" + c[10] + " burpees for " + c[11] + " seconds with " + c[12] + " second rest ")
        outputStr = (pushupStr + situpStr + squatStr + burpeeStr)
    else:
        outputStr = "fail"
    
    return outputStr
#for i in range(15) :
#        print(decodeWorkout(allWorkoutsString[i]))


In [7]:
#encoding each of the characters into integers so the model can understand -> paying special attention to the stop_sign
import pickle

with open('../input/lstmfile/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


print(tokenizer.get_config())
VOCABULARY_SIZE = len(tokenizer.word_counts) + 1 

dataset_vectorized = tokenizer.texts_to_sequences(allWorkoutsString)
print('Vectorized dataset size', len(dataset_vectorized))


for i in range(10):
    print(tokenizer.texts_to_sequences([f"{i}"]))



{'num_words': None, 'filters': '', 'lower': False, 'split': '', 'char_level': True, 'oov_token': None, 'document_count': 1500, 'word_counts': '{"5": 5707, "-": 1500, "D": 1500, "L": 1500, ":": 1500, "~": 19500, "2": 4330, "7": 1901, "3": 3542, "0": 4869, "1": 4829, "6": 2221, "4": 3507, "\\u2423": 1500, "9": 1132, "8": 1554}', 'word_docs': '{"5": 1472, "~": 1500, "L": 1500, "0": 1479, "1": 1435, "6": 1182, "-": 1500, "3": 1347, ":": 1500, "4": 1360, "2": 1406, "7": 1095, "\\u2423": 1500, "D": 1500, "9": 835, "8": 1005}', 'index_docs': '{"2": 1472, "1": 1500, "13": 1500, "3": 1479, "4": 1435, "8": 1182, "11": 1500, "6": 1347, "14": 1500, "7": 1360, "5": 1406, "9": 1095, "15": 1500, "12": 1500, "16": 835, "10": 1005}', 'index_word': '{"1": "~", "2": "5", "3": "0", "4": "1", "5": "2", "6": "3", "7": "4", "8": "6", "9": "7", "10": "8", "11": "-", "12": "D", "13": "L", "14": ":", "15": "\\u2423", "16": "9"}', 'word_index': '{"~": 1, "5": 2, "0": 3, "1": 4, "2": 5, "3": 6, "4": 7, "6": 8, "7

In [8]:
#add padding to ensure same length --- only if all are not the sane length
print(len(dataset_vectorized))
maxLength = 0
for workout_index, workout in enumerate(dataset_vectorized[:len(dataset_vectorized)]):
    #if(len(workout)) != 167:
    if(maxLength < len(workout)):
        #print('Workout #{} length: {}'.format(workout_index + 1, len(workout)))
        maxLength = len(workout)
#padding is required
print(maxLength)

1
42


In [9]:
def decodeVector(vector):
    decodedWorkout = tokenizer.sequences_to_texts([vector])[0]
    print(decodedWorkout)
    
#basic test to see if the decoder is working
decodeVector(dataset_vectorized[0])

5 - D L : ~ 5 0 ~ 5 0 ~ 2 8 ~ 1 4 ~ 1 5 ~ 1 7 ~ 1 5 ~ 1 5 ~ 3 6 ~ 3 4 ~ 3 5 ~ 5 ~ ␣


In [10]:
dataset_vectorized_padded_without_stops = tf.keras.preprocessing.sequence.pad_sequences(
    dataset_vectorized,
    padding='post',
    truncating='post',
    # We use -1 here and +1 in the next step to make sure
    # that all recipes will have at least 1 stops sign at the end,
    # since each sequence will be shifted and truncated afterwards
    # (to generate X and Y sequences).
    maxlen=maxLength-1,
    #value=tokenizer.texts_to_sequences([STOP_SIGN])[0]
)
dataset_vectorized_padded = tf.keras.preprocessing.sequence.pad_sequences(
    dataset_vectorized_padded_without_stops,
    padding='post',
    truncating='post',
    maxlen=maxLength+1,
    #value=tokenizer.texts_to_sequences([STOP_SIGN])[0]
)

for workout_index, workout in enumerate(dataset_vectorized_padded[:len(dataset_vectorized_padded)]):
    if(len(workout)) != maxLength+1:
        print('Workout #{} length: {}'.format(workout_index + 1, len(workout)))

#test it out
decodeVector(dataset_vectorized_padded[0])

5 - D L : ~ 5 0 ~ 5 0 ~ 2 8 ~ 1 4 ~ 1 5 ~ 1 7 ~ 1 5 ~ 1 5 ~ 3 6 ~ 3 4 ~ 3 5 ~ 5 ~


In [11]:
dataset = tf.data.Dataset.from_tensor_slices(dataset_vectorized_padded)

2022-04-23 21:55:03.342448: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 21:55:03.435512: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 21:55:03.436239: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-23 21:55:03.439664: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [12]:
#split processing
def split_input_target(workout):
    input_text = workout[:-1]
    target_text = workout[1:]
    
    return input_text, target_text
dataset_targeted = dataset.map(split_input_target)
print(dataset_targeted)

<MapDataset shapes: ((42,), (42,)), types: (tf.int32, tf.int32)>


In [13]:
def loss(labels, logits):
    entropy = tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )
    
    return entropy
#compile

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #used to be 0.001 which worked much better


early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=5,
    monitor='loss',
    restore_best_weights=True,
    verbose=1
)

In [14]:
# Create a checkpoints directory. TO SAVE AND CONTINUE TRAINING
checkpoint_dir = 'tmp/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_') #used to be ckpt_{epoch}, got rid due to space issues on kaggle
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

tqdm_callback = tfa.callbacks.TQDMProgressBar()

In [15]:

def loss(labels, logits):
    entropy = tf.keras.losses.sparse_categorical_crossentropy(
      y_true=labels,
      y_pred=logits,
      from_logits=True
    )
    
    return entropy


#model = keras.models.load_model(
#    "../input/lstmfile/baselineModel.h5", custom_objects={"loss": loss}
#)


model = keras.models.load_model("../input/lstmfile/simplified_model.h5")


model.compile(
    optimizer=adam_optimizer,
    loss=loss
)



In [16]:
# Batch size.
BATCH_SIZE = 1
# Buffer size to shuffle the dataset (TF data is designed to work
# with possibly infinite sequences, so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in
# which it shuffles elements).
SHUFFLE_BUFFER_SIZE = 1000
dataset_train = dataset_targeted \
  .shuffle(SHUFFLE_BUFFER_SIZE) \
  .batch(BATCH_SIZE, drop_remainder=True) \
  .repeat()
print(dataset_train)

<RepeatDataset shapes: ((1, 42), (1, 42)), types: (tf.int32, tf.int32)>


In [17]:
#const
EPOCHS = 1 #5 seemed to work well
INITIAL_EPOCH = 1
STEPS_PER_EPOCH = len(allWorkoutsString) / BATCH_SIZE
print('EPOCHS:          ', EPOCHS) 
print('INITIAL_EPOCH:   ', INITIAL_EPOCH)
print('STEPS_PER_EPOCH: ', STEPS_PER_EPOCH) #default was 1500

EPOCHS:           1
INITIAL_EPOCH:    1
STEPS_PER_EPOCH:  1.0


In [18]:
history = model.fit(
    x=dataset_train,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    initial_epoch=INITIAL_EPOCH,
    callbacks=[
        checkpoint_callback,
        #tqdm_callback
       # early_stopping_callback #we want to overfit this so get rid of early stopping
    ]
)

In [19]:
def generate_text(model, start_string, num_generate = 1000, temperature=1.0):
    # Evaluation step (generating text using the learned model)

    padded_start_string = start_string
    # Converting our start string to numbers (vectorizing).
    input_indices = np.array(tokenizer.texts_to_sequences([padded_start_string]))
    # Empty string to store our results.
    text_generated = []
    # Here batch size == 1.
   # model.reset_states()
    for char_index in range(num_generate):
        predictions = model(input_indices)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        # Using a categorical distribution to predict the character returned by the model.
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
            predictions,
            num_samples=1
        )[-1, 0].numpy()
        # We pass the predicted character as the next input to the model
        # along with the previous hidden state.
        input_indices = tf.expand_dims([predicted_id], 0)

        next_character = tokenizer.sequences_to_texts(input_indices.numpy())[0]
        text_generated.append(next_character)

       # output = ''.join(text_generated)

       # output = decodeWorkout(output)

    return (padded_start_string + ''.join(text_generated))

In [20]:
#vocab_size=VOCABULARY_SIZE
#embedding_dim=256
#rnn_units=1024
#
#simplified_batch_size = 1
#model_simplified = build_model(vocab_size, embedding_dim, rnn_units, simplified_batch_size)
#model_simplified.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
#model_simplified.build(tf.TensorShape([simplified_batch_size, None]))
#model_simplified.summary()

nonTrainedModel = keras.models.load_model("../input/lstmfile/simplified_model.h5")

In [21]:
output = (generate_text(model, '5-DL:', num_generate = maxLength, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))

2022-04-23 21:55:08.230807: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


5-DL:~10~10~44~30~30~9~74~80~0~68~75~4~␣
5-DL:
10 pushups for 10 seconds with 44 second rest 
30 situps for 30 seconds with 9 second rest 
74 squats for 80 seconds with 0 second rest 
68 burpees for 75 seconds with 4 second rest 


In [22]:
output = (generate_text(model, '4-DL:', num_generate = 43, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))

4-DL:~32~40~24~24~30~36~28~40~9~64~80~0~␣
4-DL:
32 pushups for 40 seconds with 24 second rest 
24 situps for 30 seconds with 36 second rest 
28 squats for 40 seconds with 9 second rest 
64 burpees for 80 seconds with 0 second rest 


In [23]:
output = (generate_text(model, '3-DL:', num_generate = maxLength, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))


3-DL:~13~25~24~40~70~7~12~20~14~35~60~9~␣
3-DL:
13 pushups for 25 seconds with 24 second rest 
40 situps for 70 seconds with 7 second rest 
12 squats for 20 seconds with 14 second rest 
35 burpees for 60 seconds with 9 second rest 


In [24]:
output = (generate_text(model, '2-DL:', num_generate = maxLength, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))


print(len(output))
print(maxLength)

2-DL:~8~20~32~22~65~6~5~15~34~26~75~1~␣
2-DL:
8 pushups for 20 seconds with 32 second rest 
22 situps for 65 seconds with 6 second rest 
5 squats for 15 seconds with 34 second rest 
26 burpees for 75 seconds with 1 second rest 
39
42


In [25]:
output = (generate_text(model, 'DL:', num_generate = maxLength, temperature=1.0)) #make sure you pass it model_simplified
print(output)
print(decodeWorkout(output))

DL:~8~40~23~6~35~2~11~65~8~10~60~15~␣
DL:
8 pushups for 40 seconds with 23 second rest 
6 situps for 35 seconds with 2 second rest 
11 squats for 65 seconds with 8 second rest 
10 burpees for 60 seconds with 15 second rest 
